In [1]:
from library.experiments import *
from library.objective_function import *
from library.optimiser import *

In [4]:
# init and setup one experiment
exp = single_experiment()
# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)
ad = adam()
optParas = {
         'x0': torch.tensor([16.23, 23.023], requires_grad=True),
         'alpha': 0.1,
         'beta_1': 0.9, 
         'beta_2': 0.999, 
         'epsilon': 1e-11, 
         'max_iter': 2000,
         'tol': 1e-6,              
         'verbose': True,
         'record': False }
ad.set_parameters(optParas)
exp.set_optimizer(ad)
exp.do()



*******starting optimisation from intitial point:  tensor([16.2300, 23.0230], grad_fn=<SqueezeBackward0>)
total evaluatios = 201
gradient at stop position = tensor([-0.0001,  0.0001]),
modified graident = tensor([-2.1003e-06, -8.8239e-06])
found minimum position = tensor([15.9994, 22.9992], requires_grad=True), found minimum = 19.61959457397461
Result:  local minimum
found minimum: 19.61959457397461, minimum position: [15.999424 22.999178], evals: 201


('local minimum', tensor(19.6196, grad_fn=<AddBackward0>), 201)

In [31]:
# init and setup one experiment
exp = single_experiment()
# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)
ln = line_search()
optParas = {
    'x0': torch.tensor([16.23, 23.023], requires_grad=True),
    'alpha': 1,
    'beta': 0.1, 
    'max_iter': 1000,
    'tol': 1e-3,              
    'verbose': True,
    'record': False
}
ln.set_parameters(optParas)
exp.set_optimizer(ln)
exp.do()


*******starting optimisation from intitial point:  tensor([16.2300, 23.0230], grad_fn=<SqueezeBackward0>)
total evaluatios = 41
gradient at stop position = tensor([0.0005, 0.0002])
found minimum position = tensor([15.9994, 22.9992], requires_grad=True), found minimum = 19.61959457397461
Result:  local minimum
found minimum: 19.61959457397461, minimum position: [15.999435 22.99918 ], evals: 41


('local minimum', tensor(19.6196, grad_fn=<AddBackward0>), 41)

In [26]:
# init and setup one experiment
exp = single_experiment()
# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)
opt = cma_es()
optParas ={'x0': torch.tensor([196.23, 123.023], requires_grad=True),
           'std': torch.ones((2,)) * 3, 
           'tol': 1e-3, 
           'adjust_func': do_nothing(), 
           'record': False, 
           'verbose': True}
opt.set_parameters(optParas)
exp.set_optimizer(opt)
exp.do()



*******starting optimisation from intitial mean:  tensor([196.2300, 123.0230], grad_fn=<SqueezeBackward0>)
eigenvalue of variance = tensor([9.8974e-04, 6.7251e-05], grad_fn=<SqrtBackward>)
total iterations = 400, total evaluatios = 2400
found minimum position = tensor([-2174968., -1005778.], grad_fn=<SelectBackward>), found minimum = 20.00206756591797
Result:  local minimum
found minimum: 20.00206756591797, minimum position: [-2174968. -1005778.], evals: 2400


('local minimum', tensor(20.0021, grad_fn=<SelectBackward>), 2400)

In [25]:
# init and setup one experiment
exp = single_experiment()
# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)

inner_opt = line_search()
innerOptParas = {
   'x0': None,
    'alpha': 1,
    'beta': 0.1, 
    'max_iter': 100,
    'tol': 1e-1,              
    'verbose': False,
    'record': False
}
inner_opt.set_parameters(innerOptParas)

opt = cma_es()
optParas ={'x0': torch.tensor([196.23, 123.023], requires_grad=True),
           'std': torch.ones((2,)) * 30, 
           'tol': 1e-3, 
           'adjust_func': inner_opt, 
           'record': False, 
           'verbose': True}
opt.set_parameters(optParas)
exp.set_optimizer(opt)
exp.do()



*******starting optimisation from intitial mean:  tensor([196.2300, 123.0230], grad_fn=<SqueezeBackward0>)
eigenvalue of variance = tensor([ 8.1473, 12.1323], grad_fn=<SqrtBackward>)
total iterations = 35, total evaluatios = 12136
found minimum position = tensor([ 4.1871e-07, -1.2184e-07], grad_fn=<SelectBackward>), found minimum = 9.5367431640625e-07
Result:  global minimum
found minimum: 9.5367431640625e-07, minimum position: [ 4.1870697e-07 -1.2184466e-07], evals: 12136


('global minimum', tensor(9.5367e-07, grad_fn=<SelectBackward>), 12136)

In [30]:
exp = single_experiment()
# setup objective function
ak = ackley()
exp.set_objective(ak)

inner_opt = adam()
optParas = {
         'x0': None,
         'alpha': 1,
         'beta_1': 0.9, 
         'beta_2': 0.999, 
         'epsilon': 1e-11, 
         'max_iter': 100,
         'tol': 1e-1,              
         'verbose': False,
         'record': False }
inner_opt.set_parameters(optParas)

opt = cma_es()
optParas ={'x0': torch.tensor([196.23, 123.023], requires_grad=True),
           'std': torch.ones((2,)) * 30, 
           'tol': 1e-3, 
           'adjust_func': inner_opt, 
           'record': False, 
           'verbose': True}
opt.set_parameters(optParas)
exp.set_optimizer(opt)
exp.do()



*******starting optimisation from intitial mean:  tensor([196.2300, 123.0230], grad_fn=<SqueezeBackward0>)
eigenvalue of variance = tensor([0.0018, 0.0028], grad_fn=<SqrtBackward>)
total iterations = 400, total evaluatios = 186400
found minimum position = tensor([14.0245, -5.0542], grad_fn=<SelectBackward>), found minimum = 17.66341209411621
Result:  local minimum
found minimum: 17.66341209411621, minimum position: [14.024487 -5.054191], evals: 186400


('local minimum', tensor(17.6634, grad_fn=<SelectBackward>), 186400)